In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
net=nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))

In [2]:
X=torch.rand(2,20)
net(X)#net传入的参数为一个batch的数据，所以第一维度为batch_size


tensor([[ 0.2059, -0.0123, -0.1289, -0.1425,  0.0083,  0.0968,  0.0136,  0.0825,
         -0.0387,  0.1208],
        [ 0.1916,  0.0379, -0.3555, -0.0456, -0.1162,  0.1035,  0.1035,  0.0883,
          0.0997,  0.0922]], grad_fn=<AddmmBackward0>)

In [3]:
#自定义模型
class MLP(nn.Module):#继承nn.Module
    def __init__(self):
        super().__init__()  #调用父类的构造函数
        self.hidden=nn.Linear(20,256)
        self.out=nn.Linear(256,10)
    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))

net=MLP()
net(X)

tensor([[ 0.0736, -0.0550,  0.2460, -0.0566, -0.0358, -0.0349,  0.1064,  0.1641,
          0.0566, -0.0952],
        [ 0.0278, -0.0262,  0.1940,  0.0267, -0.1388,  0.0072,  0.2290,  0.1993,
          0.0675, -0.0486]], grad_fn=<AddmmBackward0>)

In [ ]:
#顺序块
class MySequential(nn.Module):#继承nn.Module
    def __init__(self,*args):
        super().__init__()
        for block in args:#block为每个块
            self._modules[block]=block#将每个块加入到模型的_modules中，这是一个字典,可以将block看成一个层
    def forward(self,X):
        for block in self._modules.values():
            X=block(X)
        return X
net=MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

In [ ]:
#在正向传播函数中执行代码
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight=torch.rand((20,20),requires_grad=False)#不需要求梯度
        self.linear=nn.Linear(20,20)
    def forward(self,X):
        X=self.linear(X)
        X=F.relu(torch.mm(X,self.rand_weight)+1)
        X=self.linear(X)
        while X.abs().sum()>1:
            X/=2
        return X.sum()
net=FixedHiddenMLP()
net(X)